<a href="https://colab.research.google.com/github/Romulo-Anjos/Codenation_2020/blob/master/Enem2016_Codenation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Starting by importing the libraries and data provided for the exercise

In [0]:
# Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import the dataset
raw_test =  pd.read_csv('test.csv')
raw_train = pd.read_csv('train.csv')

Taking a look on the dimensions on both files:

In [162]:
print(raw_train.shape)
print(raw_test.shape)

(13730, 167)
(4576, 47)


# **1 - Data Preprocessing**

**A) Eliminating the empty data**

Checking for the Null values for the grades on the Train data

In [165]:
#Missing Data on Merge Dataset
print('Missing Data on NU_NOTA_MT:',raw_train['NU_NOTA_MT'].isnull().sum())
print('Missing Data on NU_NOTA_CN:',raw_train['NU_NOTA_CN'].isnull().sum())
print('Missing Data on NU_NOTA_CH:',raw_train['NU_NOTA_CH'].isnull().sum())
print('Missing Data on NU_NOTA_REDACAO:',raw_train['NU_NOTA_REDACAO'].isnull().sum())
print('Missing Data on NU_NOTA_LC:',raw_train['NU_NOTA_LC'].isnull().sum())

Missing Data on NU_NOTA_MT: 3597
Missing Data on NU_NOTA_CN: 3389
Missing Data on NU_NOTA_CH: 3389
Missing Data on NU_NOTA_REDACAO: 3597
ERROR! Session/line number was not unique inMissing Data on NU_NOTA_LC: 3597 
database. History logging moved to new session 61


Once the grades for Math, Writing and Languages are the same, it is possible to infer that the three tests were given at the same date. So, the null values refer to students that didn't take the test. Therefore is necessary to take off this Id's from the both datasets because there is no need to predict the Math grade (it is already 0)

In [0]:
ID = 'NU_INSCRICAO'
# Store ID's of Test Dataset which we will set the prediction result as NaN
NaNs_ID = raw_test.loc[raw_test['TP_STATUS_REDACAO'].isnull(),ID] # Saving the NaN IDs that will be added on the answer file after the prediction
raw_test = raw_test[~raw_test['TP_STATUS_REDACAO'].isnull()] #Remove those examples from test dataset
raw_train = raw_train[~raw_train['TP_STATUS_REDACAO'].isnull()] #Remove those examples from train dataset

Checking the new dimension for the Train and Test set

In [167]:
print(raw_train.shape)
print(raw_test.shape)

(10133, 167)
(3377, 47)


**B) Choosing the most important variables**

I analised the dictionary for the columns provided on the files. After analising the Dictionary of columns, I selected the most important columns to be analyzed:


In [0]:
data_train= raw_train[[#'NU_INSCRICAO',
'TP_SEXO',
'Q006',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
'NU_NOTA_MT']]
data_test= raw_test[[#'NU_INSCRICAO',
'TP_SEXO',
'Q006',
'NU_NOTA_CN',
'NU_NOTA_CH',
'NU_NOTA_LC',
]]

**C) Treating the missing data**

I will check what data I still need to treat (NaN values)

In [188]:
# Missing Data on Train Dataset
total = data_train.isnull().sum().sort_values(ascending=False)
percent = (data_train.isnull().sum()/data_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

# Missing Data on Test Dataset
total = data_test.isnull().sum().sort_values(ascending=False)
percent = (data_test.isnull().sum()/data_test.isnull().count()).sort_values(ascending=False)
missing_data_test = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


print (missing_data.head(20))
print (missing_data_test.head(20))

            Total   Percent
NU_NOTA_CH     36  0.003553
NU_NOTA_CN     36  0.003553
NU_NOTA_MT      0  0.000000
NU_NOTA_LC      0  0.000000
Q006            0  0.000000
TP_SEXO         0  0.000000
            Total   Percent
NU_NOTA_CH     17  0.005034
NU_NOTA_CN     17  0.005034
NU_NOTA_LC      0  0.000000
Q006            0  0.000000
TP_SEXO         0  0.000000


Completing the missed data with the mean values

In [0]:

# Taking care of missing data (grades)
from sklearn.impute import SimpleImputer
missingvalues = SimpleImputer(missing_values=np.nan, strategy='mean', verbose=0)
missingvalues = missingvalues.fit(data_train.iloc[:,2:6])
data_train.iloc[:,2:6]=missingvalues.transform(data_train.iloc[:,2:6])

# Taking care of missing data (grades)
from sklearn.impute import SimpleImputer
missingvalues = SimpleImputer(missing_values=np.nan, strategy='mean', verbose=0)
missingvalues = missingvalues.fit(data_test.iloc[:,2:5])
data_test.iloc[:,2:5]=missingvalues.transform(data_test.iloc[:,2:5])

Checking now the number of NaN values

In [190]:
# Missing Data on Train Dataset
total = data_train.isnull().sum().sort_values(ascending=False)
percent = (data_train.isnull().sum()/data_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

# Missing Data on Test Dataset
total = data_test.isnull().sum().sort_values(ascending=False)
percent = (data_test.isnull().sum()/data_test.isnull().count()).sort_values(ascending=False)
missing_data_test = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])


print (missing_data.head(20))
print (missing_data_test.head(20))

            Total  Percent
NU_NOTA_MT      0      0.0
NU_NOTA_LC      0      0.0
NU_NOTA_CH      0      0.0
NU_NOTA_CN      0      0.0
Q006            0      0.0
TP_SEXO         0      0.0
            Total  Percent
NU_NOTA_LC      0      0.0
NU_NOTA_CH      0      0.0
NU_NOTA_CN      0      0.0
Q006            0      0.0
TP_SEXO         0      0.0


**D) Encoding the variables**

In [191]:
qualitative_features_test = [q for q in data_test.columns 
                        if (data_test[q].dtypes == object) | (data_test[q].dtypes == bool)] #List of Features Qualitativas from test dataset

qualitative_features_train = [q for q in data_train.columns 
                        if (data_train[q].dtypes == object) | (data_train[q].dtypes == bool)] #List of Features Qualitativas from train dataset

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
d = defaultdict(LabelEncoder)
### Encoding the variables from data_test
fit = data_test[qualitative_features_test].apply(lambda x_test: d[x_test.name].fit_transform(x_test))
data_test[qualitative_features_test] = fit

### Encoding the variables from data_train
fit = data_train[qualitative_features_train].apply(lambda x_train: d[x_train.name].fit_transform(x_train))
data_train[qualitative_features_train] = fit

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


# **2 - Building the Model**

Defining X and y for data_train

In [0]:
# Defining the X and y for data_train
X = data_train.iloc[:,:-1].values
y = data_train.iloc[:,5].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

Executing the Linear Regression

In [0]:
# Fitting Multiple Linear Regression in the Training Set
from sklearn.linear_model import LinearRegression
regressor =  LinearRegression()
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)

Optimizing the optimal model using Backward Elimination

In [0]:
# Building the optimal model using Backward Elimination
import statsmodels.api as sm
X = np.append(arr = np.ones((10133,1)).astype(int), values = X, axis = 1)
X_opt = X[:,[0,1,2,3,4]]
regressor_OLS = sm.OLS(endog = y, exog= X_opt).fit()
regressor_OLS.summary()

Creating the predictions

In [0]:
prediction = regressor.predict(data_test)

Creating the file to be submitted at Codenation

In [0]:
df_saida = pd.DataFrame(
        {'NU_INSCRICAO':list(test_id)+list(NaNs_ID), #Including the IDs of empty values I took off to build the model
         'NU_NOTA_MT':list(prediction)+[np.nan]*len(NaNs_ID) # The same as the line before
                })
df_saida.to_csv(r'/content/sample_data/answer_3.csv', index=False)

Executing the Random Forest



In [0]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train,y_train)

# Predicting a new result
y_pred_random = regressor.predict(X_test)

prediction_2 = regressor.predict(data_test)

df_saida = pd.DataFrame(
        {'NU_INSCRICAO':list(test_id)+list(NaNs_ID), #Including the IDs of empty values I took off to build the model
         'NU_NOTA_MT':list(prediction_2)+[np.nan]*len(NaNs_ID) # The same as the line before
                })
df_saida.to_csv(r'/content/sample_data/answer_random.csv', index=False)